In [34]:
import numpy as np
import mosek
from itertools import permutations
from math import sqrt,factorial,log

In [35]:
def prefer(loci,locj): 
    # returns 1/2 if both candidates are preferred equally.
    # returns 1 if c_i is preferred than c_j and 0 otherwise.
    if loci==locj:
        val=1/2
    else:
        val=int(loci<locj)
    return val

def prefermatrix(rank):
    # S(t): returns the preference matrix for a given ordering.
    loc=[rank.index(i) for i in candidate]
    a=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if i!=j:
                a[i,j]= prefer(loc[i],loc[j])
    return a

def FrobeniusNorm(a):
    norm=0
    b=a.shape[0]
    for i in range(1,b):
        for j in range(i):
            norm+=2*(a[i,j])**2
    return sqrt(norm)

def entropy(xx):
    ent=0
    negative=0
    zero=0
    for i in range(T):
        if xx[i]<0:
            negative+=1
            continue
        if xx[i]==0:
            zero+=1
            continue
        ent-=xx[i]*log(xx[i])
        
    print('the number of negative values: ',negative,'; the number of 0: ',zero)
    return ent

def matrix2vector(m):
    n=m.shape[0]
    Aj=np.zeros((int(n*(n-1)/2),1))
    index=0
    for i in range(n):
        for j in range(i):
            #f i < j:
            Aj[index,0]=m[i,j]
            index+=1
    return Aj

In [36]:
def msk_relent(task, t, x, y):
    v = msk_newvar(task, 1)
    c = msk_newcon(task, 1)
    task.putaij(c, v, 1.0)
    task.putaij(c, t, 1.0)
    task.putconbound(c, mosek.boundkey.fx, 0.0, 0.0)
    task.appendcone(mosek.conetype.pexp, 0.0, [y, x, v])

def msk_newvar(task, num):  # free
    v = task.getnumvar()
    task.appendvars(num)
    for i in range(num):
        task.putvarbound(v+i, mosek.boundkey.fr, -inf, inf)
    return v

def msk_newvar_fx(task, num, val):  # fixed
    v = task.getnumvar()
    task.appendvars(num)
    for i in range(num):
        task.putvarbound(v+i, mosek.boundkey.fx, val, val)
    return v

def msk_newcon(task, num):
    c = task.getnumcon()
    task.appendcons(num)
    return c

def msk_sq(task, t, x): # t >= x^2
    task.appendcone(mosek.conetype.rquad, 0.0, [msk_newvar_fx(task, 1, 0.5), t, x])

In [19]:
def mev_shortvector(empirical,theoretical):
    ### MEV ==================================================================

    env  = mosek.Env()
    task = env.Task(0,0)

    # variables -------------------------------------------------------------
    # t is the entropy of u
    eta = msk_newvar(task, T)
    u = msk_newvar(task, T)

    # constraints ------------------------------------------------------------

    # 0<u<1 
    for i in range(T):
        task.putvarbound(u+i, mosek.boundkey.ra, 0, 1)
        task.putvarbound(eta+i , mosek.boundkey.ra, -inf, 0)

    # sum(u)=1
    c = msk_newcon(task, 1)
    task.putarow(c, range(u,u+T),[1.0]*T)
    #task.putaij(c, 1, 1.0)
    task.putconbound(c, mosek.boundkey.fx, 1.0, 1.0)    

    # max entropy
    for i in range(T):
        msk_relent(task, eta+i, u+i, msk_newvar_fx(task, 1, 1.0)) #(t,x,y) t >= x * log(x/y), x,y>=0  --> -t <= -x * log x 

    # u*theoretical=empirical
    b=matrix2vector(empirical)
    A=matrix2vector(theoretical[0])
    for k in range(1,T):
        A=np.concatenate((A,matrix2vector(theoretical[k])),axis=1)

    for row in range(A.shape[0]):
        c = msk_newcon(task, 1)
        task.putarow(c, range(u, u+T),A[row,:].tolist())
        task.putconbound(c, mosek.boundkey.fx, b[row][0], b[row][0])
            
    # objective ----------------------------------------------------------------
    task.putclist(range(eta,eta+T),[1]*T)
    #task.putclist(range(task.getnumvar()),[1.0]*task.getnumvar())

    # Input the objective sense (minimize/maximize)
    task.putobjsense(mosek.objsense.minimize)
    
    # Solve the problem and print summary
    starttime = time.time()
    task.optimize()
    endtime = time.time()

    # get solution of the diagonal (equal to the SDP var)
    xx = [0.0] * task.getnumvar()
    task.getxx(mosek.soltype.itr,xx)
    return xx[u:u+T]

In [29]:
maxval=0
minval=0.5
n=3
candidate=['c'+str(i) for i in range(1,1+n)]
inf=100

# Get all strong orderings (permutations)
perm=permutations(candidate)
T=sum(1 for ignore in perm)
# distRD[t] Gives the proportion of people who vote for ordering t.

for i in range(100):
    r=0.01*i
    distRD=[r,0,0.99-r,0,0,0.01]

    # empirical preference matrix: \hat(S)(V) 
    # theoretical preference matrices for each strong ordering.S(t)
    empirical=np.zeros((n,n))
    theoretical={}
    index=0
    for i in permutations(candidate):
        empirical+=prefermatrix(i)*distRD[index]
        theoretical[index]=prefermatrix(i)
        index+=1

    x=mev_shortvector(empirical,theoretical)
    print('the probability of c_3 winning is ',x[4]+x[5])
    if maxval < x[4]+x[5]:
        maxval=x[4]+x[5]
    if minval > x[4]+x[5]:
        minval=x[4]+x[5]

the probability of c_3 winning is  0.010000000341247824
the probability of c_3 winning is  0.005012606691198659
the probability of c_3 winning is  0.003358311717592389
the probability of c_3 winning is  0.0025359872098907
the probability of c_3 winning is  0.0020456517727023204
the probability of c_3 winning is  0.0017195770972982943
the probability of c_3 winning is  0.0014874878981104526
the probability of c_3 winning is  0.0013140522420425882
the probability of c_3 winning is  0.0011801134349079976
the probability of c_3 winning is  0.0010729399053491645
the probability of c_3 winning is  0.0009857085260037601
the probability of c_3 winning is  0.0009132418965437806
the probability of c_3 winning is  0.0008523190573595069
the probability of c_3 winning is  0.0008005751444957703
the probability of c_3 winning is  0.0007553590199679684
the probability of c_3 winning is  0.0007163028690510418
the probability of c_3 winning is  0.0006822119732731095
the probability of c_3 winning is  0.

In [27]:
maxval

0.010000000341247824

In [32]:
minval

0.00037787034169451864